# imported packages

- to make running easier, dependencies on tensorflow are replaced with pytorch
- basic packages
- data processing pacakges
- torch
- custom imports
    - models_search: this defines search space(building blocks), RNN controller and GAN design
    - funcitons.py: contains functions for training and validation of GAN and RNN controller
    - utils:
        - inception_score_torch.py: pytorch implementation of inception score
            - link: https://github.com/sbarratt/inception-score-pytorch.git
            - this is not perfect; inception score on original cifar-10 should be 11.24 ± 0.20 on 50k train images
            - however, this implementation yeilds about 10.5
        - fid_score_torch.py: pytorch implementation of Frechet inception distance score
            - link: https://github.com/mseitzer/pytorch-fid.git
        - utils.py: nothing changed

In [1]:
# basic python packages -- no need to install
import os
import time
import datasets
import datetime

# data processing packages
import numpy as np
from PIL import Image
from imageio import imsave

# torch
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
import torchvision
import torchvision.transforms as transforms

# utility packages
from tqdm import tqdm
import matplotlib.pyplot as plt

# custom imports
## hyperparameters
import cfg

## in models
import models_search

## import training functions
from functions import get_topk_arch_hidden, train_controller, train_shared

## from utils: computing inception score and fid score depends on tensorflow --> change to pytorch implementation (taegun)
from utils import inception_score_torch as ist
from utils import fid_score_torch as fst
from utils.utils import create_logger, RunningStats, save_checkpoint, set_log_dir


# author's setting
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True


# Prepare NAS

- from cfg.py: configurations (hyperparameters)
- from exps:
    - from autogan_search.sh: load parameters from this shell script

In [2]:
args = cfg.parse_args()
torch.cuda.manual_seed(args.random_seed)

# this should be set
args.exp_name = 'search_{}'.format(datetime.datetime.now())

# parameters from exps/autogan_search.sh
args.gen_bs = 128
args.dis_bs = 64
dataset = 'cifar10'
args.bottom_width = 4
img_size = 32
args.max_iter = 50000
args.gen_model = 'shared_gan'
args.dis_model = 'shared_gan'
args.latent_dim = 128
args.gf_dim = 256
args.df_dim = 128
args.g_spectral_norm = False
args.d_spectral_norm = True
args.g_lr = 0.0002
args.d_lr = 0.0002
args.beta1 = 0.0
args.beta2 = 0.9
args.init_type = 'xavier_uniform'
args.n_critic = 5
args.val_freq = 20
#args.arch = '1 0 1 1 1 0 0 1 1 1 0 1 0 3'

# Before Search.py

- used classes & functions
    - record_val function is used to record values to tensorboard (by taegun).

In [3]:
class GrowCtrler(object):
    def __init__(self, grow_step1, grow_step2):
        self.grow_step1 = grow_step1
        self.grow_step2 = grow_step2

    def cur_stage(self, search_iter):
        """
        Return current stage.
        :param epoch: current epoch.
        :return: current stage
        """
        if search_iter < self.grow_step1:
            return 0
        elif self.grow_step1 <= search_iter < self.grow_step2:
            return 1
        else:
            return 2


def create_ctrler(args, cur_stage, weights_init):
    controller = eval("models_search." + args.controller + ".Controller")(
        args=args, cur_stage=cur_stage
    ).cuda()
    controller.apply(weights_init)
    ctrl_optimizer = torch.optim.Adam(
        filter(lambda p: p.requires_grad, controller.parameters()),
        args.ctrl_lr,
        (args.beta1, args.beta2),
    )
    return controller, ctrl_optimizer


def create_shared_gan(args, weights_init):
    gen_net = eval("models_search." + args.gen_model + ".Generator")(args=args).cuda()
    dis_net = eval("models_search." + args.dis_model + ".Discriminator")(
        args=args
    ).cuda()
    gen_net.apply(weights_init)
    dis_net.apply(weights_init)
    gen_optimizer = torch.optim.Adam(
        filter(lambda p: p.requires_grad, gen_net.parameters()),
        args.g_lr,
        (args.beta1, args.beta2),
    )
    dis_optimizer = torch.optim.Adam(
        filter(lambda p: p.requires_grad, dis_net.parameters()),
        args.d_lr,
        (args.beta1, args.beta2),
    )
    return gen_net, dis_net, gen_optimizer, dis_optimizer


def weights_init(m):
    classname = m.__class__.__name__
    if classname.find("Conv2d") != -1:
        if args.init_type == "normal":
            nn.init.normal_(m.weight.data, 0.0, 0.02)
        elif args.init_type == "orth":
            nn.init.orthogonal_(m.weight.data)
        elif args.init_type == "xavier_uniform":
            nn.init.xavier_uniform(m.weight.data, 1.0)
        else:
            raise NotImplementedError(
                "{} unknown inital type".format(args.init_type)
            )
    elif classname.find("BatchNorm2d") != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0.0)
        

def record_val(args, fixed_z, gen_net: nn.Module, writer, step, clean_dir=False):
    fid_stat = './data/fid_stats_cifar10_train.npz'
    print('==== validation runs!!! ====')

    # eval mode
    gen_net = gen_net.eval()

    # generate images
    sample_imgs = gen_net(fixed_z)
    img_grid = torchvision.utils.make_grid(sample_imgs, nrow=5, normalize=True, scale_each=True)

    # get fid and inception score
    fid_buffer_dir = os.path.join(args.path_helper["sample_path"], "fid_buffer")
    os.makedirs(fid_buffer_dir, exist_ok=True)

    eval_iter = args.num_eval_imgs // args.eval_batch_size
    img_list = list()
    for iter_idx in tqdm(range(eval_iter), desc="sample images"):
        z = torch.cuda.FloatTensor(
            np.random.normal(0, 1, (args.eval_batch_size, args.latent_dim))
        )

        # Generate a batch of images
        gen_imgs = (
            gen_net(z)
            .mul_(127.5)
            .add_(127.5)
            .clamp_(0.0, 255.0)
            .permute(0, 2, 3, 1)
            .to("cpu", torch.uint8)
            .numpy()
        )
        for img_idx, img in enumerate(gen_imgs):
            file_name = os.path.join(fid_buffer_dir, f"iter{iter_idx}_b{img_idx}.png")
            imsave(file_name, img)
        img_list.extend(list(gen_imgs))

    # get inception score
    x = np.asarray(img_list)
    print(x.shape)
    trans = transforms.Compose([
        transforms.Scale(32),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])
    xs = []
    for d in x:
        d = Image.fromarray(d)
        d = trans(d)
        xs.append(torch.unsqueeze(d, 0))

    b = torch.Tensor(10000, 3, 32, 32)
    torch.cat(xs, out=b)
    mean, std = ist.get_IS(b)

    # get fid score
    fid_score = fst.get_fid(fid_buffer_dir, fid_stat)

    if clean_dir:
        os.system("rm -r {}".format(fid_buffer_dir))
    else:
        logger.info(f"=> sampled images are saved to {fid_buffer_dir}")

    writer.add_image("sampled_images", img_grid)
    writer.add_scalar("Inception_score/mean", mean, step)
    writer.add_scalar("Inception_score/std", std, step)
    writer.add_scalar("FID_score", fid_score, step)
    
    return mean, fid_score

# Search.py

- below cell have contents originally contained in search.py

In [4]:
# set writer for tensorboard
writer = SummaryWriter('./runs/')

# define nets
gen_net, dis_net, gen_optimizer, dis_optimizer = create_shared_gan(args, weights_init)

# set grow controller
grow_ctrler = GrowCtrler(args.grow_step1, args.grow_step2)

# initial
start_search_iter = 0

# set writer
if args.load_path:
    print(f"=> resuming from {args.load_path}")
    assert os.path.exists(args.load_path)
    checkpoint_file = os.path.join(args.load_path, "Model", "checkpoint.pth")
    assert os.path.exists(checkpoint_file)
    checkpoint = torch.load(checkpoint_file)
    # set controller && its optimizer
    cur_stage = checkpoint["cur_stage"]
    controller, ctrl_optimizer = create_ctrler(args, cur_stage, weights_init)

    start_search_iter = checkpoint["search_iter"]
    gen_net.load_state_dict(checkpoint["gen_state_dict"])
    dis_net.load_state_dict(checkpoint["dis_state_dict"])
    controller.load_state_dict(checkpoint["ctrl_state_dict"])
    gen_optimizer.load_state_dict(checkpoint["gen_optimizer"])
    dis_optimizer.load_state_dict(checkpoint["dis_optimizer"])
    ctrl_optimizer.load_state_dict(checkpoint["ctrl_optimizer"])
    prev_archs = checkpoint["prev_archs"]
    prev_hiddens = checkpoint["prev_hiddens"]
    args.path_helper = checkpoint["path_helper"]    
    logger.info(f"=> loaded checkpoint {checkpoint_file} (search iteration {start_search_iter})")
    logger = create_logger(args.path_helper["log_path"])
    
else:
    print(f"=> start new search process")
    # create new log dir
    assert args.exp_name
    args.path_helper = set_log_dir("logs", args.exp_name)
    logger = create_logger(args.path_helper["log_path"])
    prev_archs = None
    prev_hiddens = None

    # set controller && its optimizer
    cur_stage = 0
    controller, ctrl_optimizer = create_ctrler(args, cur_stage, weights_init)

# set up data_loader
dataset = datasets.ImageDataset(args, 2 ** (cur_stage + 3))
train_loader = dataset.train

logger.info(args)
writer_dict = {
    "writer": SummaryWriter(args.path_helper["log_path"]),
    "controller_steps": start_search_iter * args.ctrl_step,
}

g_loss_history = RunningStats(args.dynamic_reset_window)
d_loss_history = RunningStats(args.dynamic_reset_window)

# train loop
for search_iter in tqdm(range(int(start_search_iter), int(args.max_search_iter)), desc="search progress"):
    
    logger.info(f"<start search iteration {search_iter}>")
    if search_iter == args.grow_step1 or search_iter == args.grow_step2:

        # save
        cur_stage = grow_ctrler.cur_stage(search_iter)
        logger.info(f"=> grow to stage {cur_stage}")
        prev_archs, prev_hiddens = get_topk_arch_hidden(
            args, controller, gen_net, prev_archs, prev_hiddens
        )
        
        print(prev_archs)
        
        # grow section
        del controller
        del ctrl_optimizer
        controller, ctrl_optimizer = create_ctrler(args, cur_stage, weights_init)

        dataset = datasets.ImageDataset(args, 2 ** (cur_stage + 3))
        train_loader = dataset.train

    dynamic_reset = train_shared(
        args,
        gen_net,
        dis_net,
        g_loss_history,
        d_loss_history,
        controller,
        gen_optimizer,
        dis_optimizer,
        train_loader,
        prev_hiddens=prev_hiddens,
        prev_archs=prev_archs,
    )
    
    train_controller(
        args,
        controller,
        ctrl_optimizer,
        gen_net,
        prev_hiddens,
        prev_archs,
        writer_dict,
    )
    
    print('record to tensorboard')
    Iscore, Fscore = record_val(args, fixed_z, gen_net, writer, search_iter)
    gen_net.train()
    
    if dynamic_reset:
        logger.info("re-initialize share GAN")
        del gen_net, dis_net, gen_optimizer, dis_optimizer
        gen_net, dis_net, gen_optimizer, dis_optimizer = create_shared_gan(args, weights_init)

    save_checkpoint(
        {
            "cur_stage": cur_stage,
            "search_iter": search_iter + 1,
            "gen_model": args.gen_model,
            "dis_model": args.dis_model,
            "controller": args.controller,
            "gen_state_dict": gen_net.state_dict(),
            "dis_state_dict": dis_net.state_dict(),
            "ctrl_state_dict": controller.state_dict(),
            "gen_optimizer": gen_optimizer.state_dict(),
            "dis_optimizer": dis_optimizer.state_dict(),
            "ctrl_optimizer": ctrl_optimizer.state_dict(),
            "prev_archs": prev_archs,
            "prev_hiddens": prev_hiddens,
            "path_helper": args.path_helper,
        },
        False,
        args.path_helper["ckpt_path"],
    )
    
    search_iter += 1

final_archs, _ = get_topk_arch_hidden(args, controller, gen_net, prev_archs, prev_hiddens)
logger.info(f"discovered archs: {final_archs}")
writer.close()

/home/taegun/courseworks/lib/python3.8/site-packages/torch/cuda/__init__.py:106: UserWarning: 
NVIDIA RTX A6000 with CUDA capability sm_86 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_70.
If you want to use the NVIDIA RTX A6000 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))
/tmp/ipykernel_471639/866879161.py:61: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(m.weight.data, 1.0)


RuntimeError: CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.